In [ ]:
import os
os.environ["PYTORCH_JIT_USE_NNC_NOT_NVFUSER"] = "1"
from typing import List, Tuple, Optional, Union, Iterable
import warnings
import math

from beartype import beartype
import datetime
import plotly.graph_objects as go
from tqdm import tqdm
import yaml

import torch
from torch.utils.data import DataLoader
from torchvision.transforms import Compose
from e3nn import o3

from edf_interface import data
from diffusion_edf.gnn_data import FeaturedPoints
from diffusion_edf import train_utils
from diffusion_edf.trainer import DiffusionEdfTrainer
from diffusion_edf.visualize import visualize_pose
from diffusion_edf.agent import DiffusionEdfAgent

torch.set_printoptions(precision=4, sci_mode=False)

In [ ]:
import torch_cluster, torch_scatter
from edf_interface.utils.manipulation_utils import compute_pre_pick_trajectories, compute_pre_place_trajectories, compute_post_pick_trajectories, compute_post_place_trajectories

In [ ]:
device = 'cuda:0'
config_root_dir = 'configs/sapien'
testset = data.DemoDataset(dataset_dir='demo/sapien_demo_20230625')
idx = 0

In [ ]:
task_type = 'pick'
demo = testset[idx][0 if task_type == 'pick' else 1]
scene_pcd, grasp_pcd, target_poses = demo.scene_pcd, demo.grasp_pcd, demo.target_poses

pre_pick_traj = compute_pre_pick_trajectories(pick_poses=target_poses, approach_len=0.1, n_steps=20)[0]
post_pick_traj = compute_post_pick_trajectories(pick_poses=target_poses, lift_len=0.1, n_steps=20)[0]

In [ ]:
task_type = 'place'
demo = testset[idx][0 if task_type == 'pick' else 1]
scene_pcd, grasp_pcd, target_poses = demo.scene_pcd, demo.grasp_pcd, demo.target_poses

pre_place_traj = compute_pre_place_trajectories(place_poses=target_poses, scene_pcd=scene_pcd, grasp_pcd=grasp_pcd, 
                                                n_steps=20, dt=0.0001, cutoff_r=0.05, eps=0.0001)[0]
post_place_traj = compute_post_place_trajectories(place_poses=target_poses, pre_pick_trajectory=pre_pick_traj, n_steps=20, extrapolate_post_place=2.0)[0]

In [ ]:
traj = post_place_traj
data.TargetPoseDemo(scene_pcd=scene_pcd, grasp_pcd=grasp_pcd, target_poses=traj).show(point_size=5.)